- Define functions for data processing
- User setup section
- Load preprocessed data for assigning weather types
- Read ECMWF forecast data
- Associate data to WTs
- Plot the ensemble mean WT frequency during the NAM season

In [1]:
import datetime
from datetime import datetime
import glob
from netCDF4 import Dataset
import sys, traceback
import dateutil.parser as dparser
import string
from pdb import set_trace as stop
import numpy as np
import os
import mpl_toolkits
import pickle
import subprocess
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib as mpl
# import pylab as plt
import scipy
import matplotlib.path as mplPath
from matplotlib.patches import Polygon as Polygon2
from scipy.ndimage import gaussian_filter
import seaborn as sns
import sys
import scipy.spatial.qhull as qhull
import xarray as xr
import cfgrib
from pandas.tseries.offsets import MonthEnd

#### Define functions for data processing

In [4]:
# Interpolation functions
def interp_weights(xy, uv,d=2):
    tri = qhull.Delaunay(xy)
    simplex = tri.find_simplex(uv)
    vertices = np.take(tri.simplices, simplex, axis=0)
    temp = np.take(tri.transform, simplex, axis=0)
    delta = uv - temp[:, d]
    bary = np.einsum('njk,nk->nj', temp[:, :d, :], delta)
    return vertices, np.hstack((bary, 1 - bary.sum(axis=1, keepdims=True)))

def interpolate(values, vtx, wts):
    return np.einsum('nj,nj->n', np.take(values, vtx), wts)

# ###################################################
# ###################################################
def PreprocessWTdata(DailyVarsInput,                 # WT data [time,lat,lon,var]
                     RelAnnom=1,                     # calculate relative anomalies [1-yes; 0-no]
                     SmoothSigma=0,                  # Smoothing stddev (Gaussian smoothing)
                     RemoveAnnualCycl=1,             # remove annual cycle [1-yes; 0-no]
                     NormalizeData='D',              # normalize variables | options are  - 'C' - climatology
                                                     # - 'D' - daily (default)
                                                     # - 'N' - none
                     ReferencePer=None,              # period for normalizing the data
                     Normalize = None):              # mean, std, and spatial mean for normalization
    

    DailyVars = np.copy(DailyVarsInput)
    # Calculate relative anomaly
    if RelAnnom == 1:
        # we have to work with absolute values for this since we risk to divide by zero values in the climatology
        DailyVars=np.abs(DailyVars)
        if Normalize is None:
            if ReferencePer is None:
                DailyVars=(DailyVars-np.mean(DailyVars, axis=0)[None,:])/np.mean(DailyVars, axis=0)[None,:]
            else:
                DailyVars=(DailyVars-np.mean(DailyVars[ReferencePer], axis=0)[None,:])/np.mean(DailyVars[ReferencePer], axis=0)[None,:]
        else:
            # calculate anomalies with provided climatology
            DailyVars=(DailyVars - Normalize[2][None,:])/Normalize[2][None,:]

    if len(DailyVars.shape) == 3:
        DailyVars = DailyVars[:,:,:,None]
    # Spatially smooth the data
    DailyVars=gaussian_filter(DailyVars[:,:,:,:], sigma=(0,SmoothSigma,SmoothSigma,0))

    # Remove the annual cycle
    if RemoveAnnualCycl == 1:
        SpatialMeanData=pd.DataFrame(np.nanmean(DailyVars, axis=(1,2)))
        Averaged=np.roll(np.array(SpatialMeanData.rolling(window=21).mean()), -10, axis=0)
        Averaged[:10,:]=Averaged[11,:][None,:]; Averaged[-10:,:]=Averaged[-11,:][None,:]
        DailyVars=DailyVars-Averaged[:,None,None,:]

    # Normalize the data
    if NormalizeData == 'D':
        DailyVars=(DailyVars-np.mean(DailyVars, axis=(1,2))[:,None,None,:])/np.std(DailyVars, axis=(1,2))[:,None,None,:]
    elif NormalizeData == 'C':
        if Normalize is None:
            if ReferencePer is None:
                DailyVars=(DailyVars-np.mean(DailyVars, axis=(0,1,2))[None,None,None,:])/np.std(DailyVars, axis=(0,1,2))[None,None,None,:]
            else:
                DailyVars=(DailyVars-np.mean(DailyVars[ReferencePer], axis=(0,1,2))[None,None,None,:])/np.std(DailyVars[ReferencePer], axis=(0,1,2))[None,None,None,:]
        else:
            # use predefined normalization terms
            DailyVars=((DailyVars - Normalize[0][None,None,None,:]))/Normalize[1][None,None,None,:]
        DailyVars[np.isnan(DailyVars)]=0

    return DailyVars

# ===================================================================
def EucledianDistance(DailyVars,
                      rgrClustersFin,
                      MoreDistances=0):  # if this key is 1 the function will calculate additional distance metrics
    from scipy.spatial import distance
    
    SHAPE=DailyVars.shape
    Data_flatten=np.reshape(DailyVars, (SHAPE[0],SHAPE[1]*SHAPE[2]*SHAPE[3]))
    EucledianDist=np.zeros((SHAPE[0],rgrClustersFin[0].shape[0])); EucledianDist[:]=np.nan
    Correlation=np.copy(EucledianDist)
    Manhattan=np.copy(EucledianDist)
    Chebyshev=np.copy(EucledianDist)
    for dd in range(SHAPE[0]):
        EucledianDist[dd,:] = np.array([np.linalg.norm(rgrClustersFin[0][wt,:]-Data_flatten[dd,:]) for wt in range(rgrClustersFin[0].shape[0])])
        Correlation[dd,:] = np.array([np.corrcoef(rgrClustersFin[0][wt,:],Data_flatten[dd,:])[0][1] for wt in range(rgrClustersFin[0].shape[0])])
        
        if MoreDistances == 1:
            for wt in range(rgrClustersFin[0].shape[0]):
                x = Data_flatten[dd,:] #rgrClustersFin[0][wt,:]
                YY = rgrClustersFin[0][wt,:] #+np.random.rand(len(rgrClustersFin[0][wt,:]))
                XX = Data_flatten[dd,:]

                # ----- Manhattan Distance ------
                # Quoting from the paper, “On the Surprising Behavior of Distance Metrics in High Dimensional Space”, by Charu C. Aggarwal, Alexander 
                # Hinneburg, and Daniel A. Kiem. “ for a given problem with a fixed (high) value of the dimensionality d, it may be preferable to use 
                # lower values of p. This means that the L1 distance metric (Manhattan Distance metric) is the most preferable for high dimensional applications.”
                Manhattan[dd,wt] = distance.cityblock(XX, YY)
                Chebyshev[dd,wt] = distance.chebyshev(XX, YY)
    
    return EucledianDist, Correlation, Manhattan, Chebyshev

#### User setup section

In [5]:
RegName = 'Arizona West' # can be 'Arizona West', 'Arizona East', 'NM North', and 'NM South'
#RegName = 'Arizona East'
#RegName = 'NM North'
#RegName = 'NM South'
MONTHS = [6,7,8,9,10]    # months within the NAM season

#### Load preprocessed data for assigning weather types

In [6]:
# Setup clustering algorithm
ClusterMeth='HandK'  # current options are ['HandK','hdbscan']
ClusterBreakup = 0   # breakes up clusters that are unproportionally large (only for hdbscan)
RelAnnom=1           # 1 - calculates daily relative anomalies
NormalizeData='C'    # normalize variables | options are  - 'C' - climatology
                                                        # - 'D' - daily (default)
                                                        # - 'N' - none
MinDistDD=1          # minimum nr of days between XWT events
RemoveAnnualCycl=0   # remove annual cycle in varaiables with 21 day moving average filter

dir_data = '/glade/campaign/mmm/c3we/mingge/COEXIST/AZ_S2S_DATA/'
if RegName == 'Arizona West':
    # cluster 1501
    sCentroidFile = dir_data + '1501_XWT-centroids_train-1982-2018_eval-1982-2018_E13514_XWTs3_Vars-Q850_M-6-7-8-9-10.nc'
    sERA_data = dir_data + 'ERA-Interim_PRISM_data13514_1501_1982-2018_Q850_JJASO.npz'
    sWTdata = dir_data + 'Clusters13514_1501_1982-2018_Q850_JJASO'
    NormFact = dir_data + 'Normalization_Factors_IFS_1501.npz'
    WTsort = [0,2,1]
    re = 0
if RegName == 'Arizona East':
    # cluster 1502
    sERA_data = dir_data + 'ERA-Interim_PRISM_data13514_1502_1982-2018_Q850_JJASO.npz'
    sCentroidFile = dir_data +'1502_XWT-centroids_train-1982-2018_eval-1982-2018_E13514_XWTs3_Vars-Q850_M-6-7-8-9-10.nc'
    sWTdata = dir_data + 'Clusters13514_1502_1982-2018_Q850_JJASO'
    NormFact = dir_data + 'Normalization_Factors_IFS_1502.npz'
    WTsort = [1,2,0]
    re = 1
if RegName == 'NM North':
    # cluster HUC6-00
    sERA_data = dir_data + 'ERA-Interim_PRISM_data13514_HUC6-00_1982-2018_Q850_JJASO.npz'
    sCentroidFile = dir_data + 'HUC6-00_XWT-centroids_train-1982-2018_eval-1982-2018_E13514_XWTs3_Vars-Q850_M-6-7-8-9-10.nc'
    sWTdata = dir_data + 'Clusters13514_HUC6-00_1982-2018_Q850_JJASO'
    NormFact = dir_data + 'Normalization_Factors_IFS_HUC6-00.npz'
    WTsort = [1,2,0]
    re = 2
if RegName == 'NM South':
    # cluster HUC6-03
    sERA_data = dir_data + 'ERA-Interim_PRISM_data13514_HUC6-03_1982-2018_Q850_JJASO.np'
    sCentroidFile = dir_data + 'HUC6-03_XWT-centroids_train-1982-2018_eval-1982-2018_E13514_XWTs3_Vars-Q850_M-6-7-8-9-10.nc'
    sWTdata = dir_data + 'Clusters13514_HUC6-03_1982-2018_Q850_JJASO'
    NormFact = dir_data + 'Normalization_Factors_IFS_HUC6-03.npz'
    WTsort = [0,1,2]
    re = 3

In [8]:
# Load the Centroids
print('Centroids:',sWTdata)
with open(sWTdata, 'rb') as handle:
    npzfile = pickle.load(handle)
WTclusters=npzfile['grClustersFin']['Full']
WTlat=npzfile['LatWT']#; rgrLatT1 = WTlat
WTlon=npzfile['LonWT']#; rgrLonT1 = WTlon
# WTlon[WTlon<0] = WTlon[WTlon<0]+360
WTtime=npzfile['rgdTime']
SpatialSmoothing=npzfile['SpatialSmoothing']
WTlat.shape 

Centroids: /glade/campaign/mmm/c3we/mingge/COEXIST/AZ_S2S_DATA/Clusters13514_1501_1982-2018_Q850_JJASO


(18, 17)

In [10]:
# read ECMWF coordinates
ncid=Dataset('/glade/campaign/mmm/c3we/ECMWF/20050601/Q_GDS0_ISBL/Q_GDS0_ISBL_day_ECMWF_mem01_20050601.nc', mode='r')
rgrLonS=np.squeeze(ncid.variables['g0_lon_4'][:])
rgrLatS=np.squeeze(ncid.variables['g0_lat_3'][:])
ncid.close()

rgrLonSF, rgrLatSF = np.meshgrid(rgrLonS, rgrLatS)
rgrLonSF[rgrLonSF>180] = rgrLonSF[rgrLonSF>180]-360

# reverse Lat: make it from south to North
rgrLatSF = rgrLatSF[::-1] # IFS lat runs from N to S
 
# get the region of interest
iAddCells= 4 # grid cells added to subregion
iWest=np.argmin(np.abs(WTlon.min() - rgrLonSF[0,:]))-iAddCells
iEast=np.argmin(np.abs(WTlon.max() - rgrLonSF[0,:]))+iAddCells
iNort=np.argmin(np.abs(WTlat.max() - rgrLatSF[:,0]))+iAddCells
iSouth=np.argmin(np.abs(WTlat.min() - rgrLatSF[:,0]))-iAddCells

rgrLonS=rgrLonSF[iSouth:iNort,iWest:iEast]
rgrLatS=rgrLatSF[iSouth:iNort,iWest:iEast]

# create gregridding weights
# Remap ECMWF to ERA5
# WTlon.flatten() -> WTlon.flatten()[:,None]
#          (306,) -> (306, 1)
# wts.shape (306, 3)
points=np.array([rgrLonS.flatten(), rgrLatS.flatten()]).transpose()
vtx, wts = interp_weights(points, np.append(WTlon.flatten()[:,None], WTlat.flatten()[:,None], axis=1))

/glade/derecho/scratch/mingge/tmp/ipykernel_12711/1862231345.py:3: DeprecationWarning: Please use `Delaunay` from the `scipy.spatial` namespace, the `scipy.spatial.qhull` namespace is deprecated.
  tri = qhull.Delaunay(xy)


#### Read Normalization data

In [47]:
NormData = np.load(NormFact, allow_pickle=True)
NormData = NormData['grNormalizationFactors']

#### Read ECMWF forecast data

In [49]:
# get ECMWF_forecast.grib DATA array (202208)
year_s = 1993
year_e = 2021
n_fcst = 215

dir_i = '/glade/campaign/mmm/c3we/ECMWF/'
dir_o = '/glade/campaign/mmm/c3we/mingge/COEXIST/ECMWF/WT-frequency/'

for year in range(year_s, year_e +1):
    if year <= 2017:
        n_mem = 25
    else:
        n_mem = 51
    # Erin's note P12    
    # https://docs.google.com/presentation/d/1f9ph9H9_BBwp3iqjmEVue8cHSPNJOlAg0kDyWFvXk_s/edit#slide=id.g20ece04bfab_0_96
    for month in range(4,9,1):
    #for month in range(2,9) ??? according to Amomalies file
        yyyymm_s = str(year*100 + month)
        StartDay = datetime(year, month, 1,  0) 
        Time_datetime = pd.date_range(StartDay, periods = n_fcst, freq='d')
        #print(Time_datetime )
        NormData_m = list(NormData.item()[str(Time_datetime[0].month)]) 
        for mem in range(1, n_mem + 1):
            mem_s = str(mem).zfill(2) 
            flnm = dir_i + yyyymm_s + '/qall_GDS0_ISBL/q850_day_ECMWF_mem' + mem_s + '_'+ yyyymm_s + '.nc'
            #print(flnm,str(Time_datetime[0].month))
            with  xr.open_dataset(flnm) as ds:
                q850 = ds.Q_GDS0_ISBL
            if mem == 1:
                data_4d = q850.copy()
            else:
                data_4d = xr.concat([data_4d, q850], dim='ensemble0')
                
        DATA = (data_4d.values).squeeze()
        DATA = DATA[:,:,::-1,:]
        DATA = DATA[:,:,iSouth:iNort,iWest:iEast]
        DATA = (DATA[:,1::2]+DATA[:,::2])/2 
    
        iTime = np.isin(Time_datetime.month, MONTHS)
        Data_on_centroid_grid = np.zeros((sum(iTime), WTlon.shape[0], WTlat.shape[1],1, DATA.shape[0])); Data_on_centroid_grid[:] = np.nan

        for en in range(DATA.shape[0]):
            iTime = np.isin(Time_datetime.month, MONTHS)
            #print(iTime)
            DataAct = DATA[en,iTime,:,:]
            for tt in range(DataAct.shape[0]):
                valuesi=interpolate(DataAct[tt,:,:].flatten(), vtx, wts)
                Data_on_centroid_grid[tt,:,:,0,en] = valuesi.reshape(WTlon.shape[0],WTlat.shape[1])

        # Associiate dta to WTs
        SHAPE = Data_on_centroid_grid.shape
        WT_NMME = np.zeros((SHAPE[0],SHAPE[4])); WT_NMME[:] = np.nan

        for en in range(Data_on_centroid_grid.shape[4]):
            if np.isnan(np.nanmean(Data_on_centroid_grid[:,:,:,:,en])) == False:
                isNAN = np.isnan(np.nanmean(Data_on_centroid_grid[:,:,:,:,en], axis=(1,2,3)))
                DailyVarsEvalNorm=PreprocessWTdata(Data_on_centroid_grid[:,:,:,:,en],               # WT data [time,lat,lon,var]
                            RelAnnom=RelAnnom,                                              # calculate relative anomalies [1-yes; 0-no]
                            SmoothSigma=0,                                                  # Smoothing stddev (Gaussian smoothing)
                            RemoveAnnualCycl=RemoveAnnualCycl,                              # remove annual cycle [1-yes; 0-no]
                            NormalizeData=NormalizeData,                                    # normalize data [1-yes; 0-no]
                            Normalize = NormData_m)                                           # predefined mean and std for normalization  

                EucledianDist, Correlation, Manhattan, Chebyshev = EucledianDistance(DailyVarsEvalNorm,
                                                      WTclusters)
                EucledianDist_orig=np.copy(EucledianDist)
                EucledianDist=np.nanmin(EucledianDist,axis=1)
                MinED =  np.nanargmin(EucledianDist_orig,axis=1).astype(float)
                MinED[isNAN]  = np.nan
                WT_NMME[:,en] = MinED

        # Re-label the WTs so that: 0=Monsoon, 1=Normal, 2=Dry
        WT_NMME_FIN = np.copy(WT_NMME); WT_NMME_FIN[:] = 999
        for wt in range(len(WTsort)):
            WT_NMME_FIN[WT_NMME == WTsort[wt]] = int(wt)
            
        ### Process the output into monthly mean WT frequencies and save them into a csv file
        WT_MM = np.zeros((5, 3, WT_NMME_FIN.shape[1]))
        rgiDDmon = [30,31,31,30,31]
        for wt in range(3):
            for mm in range(5):
                WT_MM[mm,wt,:] = np.sum(WT_NMME_FIN[int(np.sum(rgiDDmon[:mm])):int(np.sum(rgiDDmon[:mm+1]))] == wt, axis=0)    
                
        rgdTimeMM = pd.date_range(Time_datetime[0], end=Time_datetime[-1], freq='m')
        rgdTimeMM = rgdTimeMM[np.isin(rgdTimeMM.month, MONTHS)]
        
        YYYYMM = [int(rgdTimeMM.year[0]*100+rgdTimeMM.month[mm]) for mm in range(len(rgdTimeMM))]
        DATA = np.round(np.mean(WT_MM[:,:,:], axis=2),2)
        grDATA = {}
        #grDATA['region'] = RegName.replace(' ','-')
        #grDATA['lead'] = Time_datetime[0].month
        grDATA['YYYYMM'] = YYYYMM
        grDATA['dry']  = DATA[:len(YYYYMM),2].astype(float)
        grDATA['normal']  = DATA[:len(YYYYMM),1].astype(float)
        grDATA['monsoon']  = DATA[:len(YYYYMM),0].astype(float)
        df = pd.DataFrame(grDATA)
        yyyymm_ini = str(Time_datetime[0].year*100 + Time_datetime[0].month)
        flnm_o = dir_o + 'ECMWF_WT-frequency_'+ RegName.replace(' ','-') + '-'+ yyyymm_ini +'_ensemble-mean.csv'
        print(flnm_o)
        df.to_csv(flnm_o, index=False)
print('finish')

/glade/campaign/mmm/c3we/mingge/COEXIST/ECMWF/WT-frequency/ECMWF_WT-frequency_NM-South-199304_ensemble-mean.csv
/glade/campaign/mmm/c3we/mingge/COEXIST/ECMWF/WT-frequency/ECMWF_WT-frequency_NM-South-199305_ensemble-mean.csv
/glade/campaign/mmm/c3we/mingge/COEXIST/ECMWF/WT-frequency/ECMWF_WT-frequency_NM-South-199306_ensemble-mean.csv
/glade/campaign/mmm/c3we/mingge/COEXIST/ECMWF/WT-frequency/ECMWF_WT-frequency_NM-South-199307_ensemble-mean.csv
/glade/campaign/mmm/c3we/mingge/COEXIST/ECMWF/WT-frequency/ECMWF_WT-frequency_NM-South-199308_ensemble-mean.csv
/glade/campaign/mmm/c3we/mingge/COEXIST/ECMWF/WT-frequency/ECMWF_WT-frequency_NM-South-199404_ensemble-mean.csv
/glade/campaign/mmm/c3we/mingge/COEXIST/ECMWF/WT-frequency/ECMWF_WT-frequency_NM-South-199405_ensemble-mean.csv
/glade/campaign/mmm/c3we/mingge/COEXIST/ECMWF/WT-frequency/ECMWF_WT-frequency_NM-South-199406_ensemble-mean.csv
/glade/campaign/mmm/c3we/mingge/COEXIST/ECMWF/WT-frequency/ECMWF_WT-frequency_NM-South-199407_ensemble-m